# Collect Movie Metadata

In [2]:
from time import time
from multiprocessing import Pool, cpu_count
import tqdm  # show multi-processing progress bar

import pandas as pd

import mojo_api

# for auto reload a module after editing.
%load_ext autoreload
%autoreload 2

### Step 1: Collect list of movies

In [2]:
# Collect movie list for each year from https://www.boxofficemojo.com/.
df = pd.DataFrame()
year_start = 2000
year_end = 2019
for year in range(year_start - 1, year_end + 1):
    year_df = mojo_api.get_movie_list(year)
    year_df.to_csv('./data/movie_list/_movie_list_{}.csv'.format(year), index=False)
    print("Finished collecting movie list of {}, movie number: {}".format(year, year_df.shape[0]))
    df = pd.concat([df, year_df])

Finished collecting movie list of 1999, movie number: 440
Finished collecting movie list of 2000, movie number: 429
Finished collecting movie list of 2001, movie number: 405
Finished collecting movie list of 2002, movie number: 553
Finished collecting movie list of 2003, movie number: 636
Finished collecting movie list of 2004, movie number: 654
Finished collecting movie list of 2005, movie number: 628
Finished collecting movie list of 2006, movie number: 712
Finished collecting movie list of 2007, movie number: 750
Finished collecting movie list of 2008, movie number: 708
Finished collecting movie list of 2009, movie number: 625
Finished collecting movie list of 2010, movie number: 615
Finished collecting movie list of 2011, movie number: 696
Finished collecting movie list of 2012, movie number: 766
Finished collecting movie list of 2013, movie number: 802
Finished collecting movie list of 2014, movie number: 808
Finished collecting movie list of 2015, movie number: 803
Finished colle

In [5]:
# A movie title can appear across two years. Remove duplicate records across years.
df_remove_dup = df.groupby('Title').agg({
    'Year': 'first',
    'Rank': 'first',
    'Title': 'first'
})

# Only keep the high rank movies.
df_remove_dup = df_remove_dup.sort_values(['Year', 'Rank'])
df_remove_dup_p1 = df_remove_dup[df_remove_dup.Year < 2014]
df_remove_dup_p1 = df_remove_dup_p1[df_remove_dup_p1.Rank <= 80]
df_remove_dup_p2 = df_remove_dup[df_remove_dup.Year >= 2014]
df_remove_dup_p2 = df_remove_dup_p2[df_remove_dup_p.Rank <= 150]

df_remove_dup = pd.concat([df_remove_dup_p1, df_remove_dup_p2])
df_remove_dup = df_remove_dup.sort_values(['Year', 'Rank'])
df_remove_dup = df_remove_dup.reset_index(drop=True)
df_remove_dup.to_csv('./data/movie_list/_movie_list_2000-2019.csv', index=False)

### Step 2: Collect metadata of each movie

In [31]:
# First process movies after 2014
df = pd.read_csv('./data/movie_list/_movie_list_2000-2019.csv')
df = df[df.Year >= 2014]
df['tt_id'] = None
df['rl_id'] = None
df['release_date'] = None
df['company'] = None
df['mpaa'] = None
df['genres'] = None
df['runtime'] = None
df['director'] = None
df['actors'] = None
df['budget'] = None
df['bo_opening'] = None
df['bo_gross'] = None
df['imdb_score'] = None

def add_info_chunk(df):
    for i in range(df.shape[0]):
        title = df['Title'].iloc[i]
        year = df['Year'].iloc[i]
        movie = mojo_api.Movie()
        movie.get_train_movie_info(title, year)
        df.loc[i, 'tt_id'] = movie.tt_id
        df.loc[i, 'rl_id'] = movie.rl_id
        df.loc[i, 'release_date'] = movie.release_date
        df.loc[i, 'company'] = movie.company
        df.loc[i, 'mpaa'] = movie.mpaa
        df.loc[i, 'genres'] = movie.genres
        df.loc[i, 'runtime'] = movie.movie_length
        df.loc[i, 'director'] = movie.director
        df.loc[i, 'actors'] = movie.actors
        df.loc[i, 'budget'] = movie.budget
        df.loc[i, 'bo_opening'] = movie.bo_opening
        df.loc[i, 'bo_gross'] = movie.bo_gross
        df.loc[i, 'imdb_score'] = movie.imdb_score
#     print(i, movie.title)
    return df

time_start = time()
top = 150
with Pool(cpu_count()) as p:
    total = top #df.shape[1]
    chunks = list(tqdm.tqdm(
                p.imap(add_info_chunk, [df[df['Rank'] == i].reset_index(drop=True) for i in range(1, top+1)]), total=total))
time_end = time()
time_run = (time_end - time_start) / 60
print("Finished. Time: {0:.1f} min.".format(time_run))

df_filled = pd.DataFrame()
for chunk in chunks:
    df_filled = pd.concat([df_filled, chunk])
df_filled = df_filled.sort_values(['Year', 'Rank'])
df_filled = df_filled.reset_index(drop=True)
df_filled = df_filled[~df_filled['tt_id'].isnull()]
df_filled = df_filled[~df_filled['rl_id'].isnull()]
df_filled = df_filled[~df_filled['budget'].isnull()]
df_filled = df_filled[~df_filled['release_date'].isnull()]
df_filled = df_filled[~df_filled['bo_opening'].isnull()]
df_filled = df_filled[~df_filled['bo_gross'].isnull()]
df_filled.to_csv('./data/movie_list/metadata_2014-2019.csv', index=False)





  0%|          | 0/150 [00:00<?, ?it/s]



  1%|          | 1/150 [00:27<1:08:23, 27.54s/it]



  2%|▏         | 3/150 [00:27<47:19, 19.32s/it]  

imdb_id not found. title: Star Wars: Episode IX - The Rise of Skywalker, year: 2019.






  3%|▎         | 5/150 [00:49<40:40, 16.83s/it]



  5%|▍         | 7/150 [00:51<28:33, 11.98s/it]



  6%|▌         | 9/150 [00:59<22:47,  9.70s/it]



  7%|▋         | 10/150 [01:10<23:04,  9.89s/it]



  8%|▊         | 12/150 [01:12<16:41,  7.26s/it]



  9%|▊         | 13/150 [01:22<18:19,  8.03s/it]



  9%|▉         | 14/150 [01:24<14:08,  6.24s/it]



 10%|█         | 15/150 [01:29<13:07,  5.83s/it]



 11%|█         | 16/150 [01:29<09:21,  4.19s/it]



 11%|█▏        | 17/150 [01:46<17:32,  7.91s/it]



 13%|█▎        | 19/150 [01:52<14:12,  6.51s/it]



 14%|█▍        | 21/150 [02:02<13:05,  6.09s/it]



 15%|█▍        | 22/150 [02:03<09:25,  4.42s/it]



 15%|█▌        | 23/150 [02:09<10:05,  4.77s/it]

imdb_id not found. title: The Angry Birds Movie, year: 2016.
imdb_id not found. title: The Upside, year: 2019.






 17%|█▋        | 25/150 [02:21<10:48,  5.19s/it]



 19%|█▊        | 28/150 [02:23<07:53,  3.88s/it]



 19%|█▉        | 29/150 [02:39<14:54,  7.39s/it]



 21%|██▏       | 32/150 [02:40<10:22,  5.28s/it]



 22%|██▏       | 33/150 [02:54<15:18,  7.85s/it]



 23%|██▎       | 34/150 [03:00<14:01,  7.25s/it]

...

Run the Race : budget still not found on IMDB.






 98%|█████████▊| 147/150 [10:16<00:10,  3.58s/it]

Island of Lemurs: Madagascar : budget still not found on IMDB.
I'll See You in My Dreams : budget still not found on IMDB.
The Wife : budget still not found on IMDB.
Beatriz at Dinner : budget still not found on IMDB.






 99%|█████████▊| 148/150 [10:29<00:12,  6.39s/it]



100%|██████████| 150/150 [10:30<00:00,  4.20s/it]


Finished. Time: 10.5 min.


In [32]:
# Then process movies before 2014
year_start = 2010
year_end = 2014
df = pd.read_csv('./data/movie_list/_movie_list_2000-2019.csv')
df = df[df.Year >= year_start]
df = df[df.Year < year_end]
df['tt_id'] = None
df['rl_id'] = None
df['release_date'] = None
df['company'] = None
df['mpaa'] = None
df['genres'] = None
df['runtime'] = None
df['director'] = None
df['actors'] = None
df['budget'] = None
df['bo_opening'] = None
df['bo_gross'] = None
df['imdb_score'] = None

def add_info_chunk(df):
    for i in range(df.shape[0]):
        title = df['Title'].iloc[i]
        year = df['Year'].iloc[i]
        movie = mojo_api.Movie()
        movie.get_train_movie_info(title, year)
        df.loc[i, 'tt_id'] = movie.tt_id
        df.loc[i, 'rl_id'] = movie.rl_id
        df.loc[i, 'release_date'] = movie.release_date
        df.loc[i, 'company'] = movie.company
        df.loc[i, 'mpaa'] = movie.mpaa
        df.loc[i, 'genres'] = movie.genres
        df.loc[i, 'runtime'] = movie.movie_length
        df.loc[i, 'director'] = movie.director
        df.loc[i, 'actors'] = movie.actors
        df.loc[i, 'budget'] = movie.budget
        df.loc[i, 'bo_opening'] = movie.bo_opening
        df.loc[i, 'bo_gross'] = movie.bo_gross
        df.loc[i, 'imdb_score'] = movie.imdb_score
#     print(i, movie.title)
    return df

time_start = time()
top = 80
with Pool(cpu_count()) as p:
    total = top #df.shape[1]
    chunks = list(tqdm.tqdm(
                p.imap(add_info_chunk, [df[df['Rank'] == i].reset_index(drop=True) for i in range(1, top+1)]), total=total))
time_end = time()
time_run = (time_end - time_start) / 60
print("Finished. Time: {0:.1f} min.".format(time_run))

df_filled = pd.DataFrame()
for chunk in chunks:
    df_filled = pd.concat([df_filled, chunk])
df_filled = df_filled.sort_values(['Year', 'Rank'])
df_filled = df_filled.reset_index(drop=True)
df_filled = df_filled[~df_filled['tt_id'].isnull()]
df_filled = df_filled[~df_filled['rl_id'].isnull()]
df_filled = df_filled[~df_filled['budget'].isnull()]
df_filled = df_filled[~df_filled['release_date'].isnull()]
df_filled = df_filled[~df_filled['bo_opening'].isnull()]
df_filled = df_filled[~df_filled['bo_gross'].isnull()]
df_filled.to_csv('./data/movie_list/metadata_{}-{}.csv'.format(year_start, year_end-1), index=False)





  0%|          | 0/80 [00:00<?, ?it/s]

imdb_id not found. title: Iron Man 3, year: 2013.






  1%|▏         | 1/80 [00:11<15:13, 11.56s/it]



  2%|▎         | 2/80 [00:17<12:56,  9.96s/it]



  4%|▍         | 3/80 [00:17<09:00,  7.02s/it]



  6%|▋         | 5/80 [00:27<07:53,  6.31s/it]



  8%|▊         | 6/80 [00:29<06:14,  5.06s/it]

imdb_id not found. title: Fast & Furious 6, year: 2013.






  9%|▉         | 7/80 [00:33<05:49,  4.78s/it]



 11%|█▏        | 9/80 [00:43<05:41,  4.82s/it]



 12%|█▎        | 10/80 [00:47<05:15,  4.51s/it]



 15%|█▌        | 12/80 [00:49<04:03,  3.58s/it]

...

imdb_id not found. title: Beauty and the Beast  2012 3D Release, year: 2012.






 85%|████████▌ | 68/80 [03:46<00:39,  3.28s/it]



 89%|████████▉ | 71/80 [03:51<00:24,  2.76s/it]



 90%|█████████ | 72/80 [03:52<00:17,  2.13s/it]



 91%|█████████▏| 73/80 [03:57<00:21,  3.06s/it]



 94%|█████████▍| 75/80 [04:00<00:12,  2.56s/it]



 95%|█████████▌| 76/80 [04:01<00:08,  2.08s/it]

imdb_id not found. title: Star Wars: Episode I - The Phantom Menace  2012 3D Release, year: 2012.






 98%|█████████▊| 78/80 [04:06<00:04,  2.32s/it]



100%|██████████| 80/80 [04:09<00:00,  3.12s/it]

Finished. Time: 4.2 min.


In [3]:
df_p1 = pd.read_csv('./data/movie_list/metadata_2000-2004.csv')
df_p2 = pd.read_csv('./data/movie_list/metadata_2005-2009.csv')
df_p3 = pd.read_csv('./data/movie_list/metadata_2010-2013.csv')
df_p4 = pd.read_csv('./data/movie_list/metadata_2014-2019.csv')
df = pd.concat([df_p1, df_p2, df_p3, df_p4])
df = df.sort_values(['Year', 'Rank'])
df = df.reset_index(drop=True)
df.to_csv('./data/movie_list/metadata_2000-2019.csv', index=False)